# PROSPACE ASSIGNMENT SUBMISSION 

We first define a way to generate some random points - 1000 as mentioned in the problem statement then figure out a transformation for the parametric cone coordinates to cartesian ones for visualisations. 
- Using np.seed(0) for reproduciblity for sampling from the unifrom distribution 
- rand_pt_generator : (r,t,p) -> (x,y,z) using appropriate transformation
- np.random.uniform(min_val , max_val) for assigning some value to the parameters 
- Repeat it for 1000 times.

In [26]:
import numpy as np 
# This function generates a random point in 3D space
# (r, t, p) are parametric coordinates -> r is the radius , t is the planer angle and p is the azimutuhal
# The function returns a tuple (x, y, z) which are the cartesian coordinates in 3D space with appropriate units
def rand_pt_generator(r, t, p):
    return (r * np.cos(t), r * np.sin(t) , r/np.tan(p))
# BEGIN: Generate 1000 points on the cone
Cartesian_Space = []
np.random.seed(0)  # for reproducibility
for i in range(1000):
    r = np.random.uniform(0, 2)  # radius
    t = np.random.uniform(0, 2*np.pi)  # planer angle
    p = np.random.uniform(np.pi/3, np.pi/4)  # azimutuhal angle
    Cartesian_Space.append(rand_pt_generator(r, t, p))
# END: Generate 1000 points on the cone


     

#### Now using Plotly.js for beautiful and clean visualisations of converted cartesian coordinates as a form of 3D scatter plot. 


In [27]:
import plotly.graph_objects as go

# Extract x, y, z coordinates from Cartesian_Space
x = [pt[0] for pt in Cartesian_Space]
y = [pt[1] for pt in Cartesian_Space]
z = [pt[2] for pt in Cartesian_Space]

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=2))])
fig.show()


##### Creating a Noise distribution to simulate real life scenarios that might occur including but not limited to 
- Turbulence
- Intra and inter particle interactions
- Video/image abberations distortions 
- Improper conversion of Video feed to 3D data 

Now we must keep in mind that the noise however random must not completely blow our model out of proportions so a thershold for its gaussian distribution must be set inorder for our model and simulated data to be accurate and usable 

In [29]:
# Assuming Cartesian_Space is your array of coordinates
Cartesian_Space = np.array(Cartesian_Space)

# Define the z-plane above which you want to add noise
z_plane = 0

# Define the mean and standard deviation of the Gaussian noise
mean = [0,0,0]
std_dev = [0.1,0.1,0.1]

# Generate the Gaussian noise for x, y, z
noise_x = np.random.normal(mean[0], std_dev[0], size=Cartesian_Space[:, 0].shape)
noise_y = np.random.normal(mean[1], std_dev[1], size=Cartesian_Space[:, 1].shape)
noise_z = np.random.normal(mean[2], std_dev[2], size=Cartesian_Space[:, 2].shape)

# Add the noise to the coordinates that are above the z-plane
Cartesian_Space_noisy = Cartesian_Space.copy()
Cartesian_Space_noisy[Cartesian_Space[:, 2] > z_plane, 0] += noise_x[Cartesian_Space[:, 2] > z_plane]
Cartesian_Space_noisy[Cartesian_Space[:, 2] > z_plane, 1] += noise_y[Cartesian_Space[:, 2] > z_plane]
Cartesian_Space_noisy[Cartesian_Space[:, 2] > z_plane, 2] += noise_z[Cartesian_Space[:, 2] > z_plane]

# Create a 3D scatter plot of the original data
fig = go.Figure(data=[go.Scatter3d(x=Cartesian_Space_noisy[:, 0], y=Cartesian_Space_noisy[:, 1], z=Cartesian_Space_noisy[:, 2], mode='markers', marker=dict(size=2))])
                
fig.show()

We observe that as the vertical height of cone increases the chaotic behavior of particles also increases which is indictive of real world scenarios as per my observations and research into this topic. 

Moving on now we must try to fit a conical surface onto this final noisy data we will be using a classical algorithm with a least square optimised loss function.  

In [30]:
from scipy.optimize import least_squares

# Define the residuals function for the cone equation
def residuals(params, x, y, z):
    a, b, c, r = params
    return z - c - r * np.sqrt((x - a)**2 + (y - b)**2)

# Initial guess for the parameters
initial_params = [0, 0, 0, 1]

# Fit the cone to the data
result = least_squares(residuals, initial_params, args=(Cartesian_Space_noisy[:, 0], Cartesian_Space_noisy[:, 1], Cartesian_Space_noisy[:, 2]))

# The optimized parameters
a, b, c, r = result.x

# Define the range and step size for the x and y coordinates
x_range = np.linspace(Cartesian_Space_noisy[:, 0].min(), Cartesian_Space_noisy[:, 0].max(), 100)
y_range = np.linspace(Cartesian_Space_noisy[:, 1].min(), Cartesian_Space_noisy[:, 1].max(), 100)

# Create a meshgrid for the x and y coordinates
x_mesh, y_mesh = np.meshgrid(x_range, y_range)

# Calculate the corresponding z coordinates using the cone equation
z_min = Cartesian_Space_noisy[:, 2].min()
z_max = Cartesian_Space_noisy[:, 2].max()
z_mesh = np.clip(c + r * np.sqrt((x_mesh - a)**2 + (y_mesh - b)**2), z_min, z_max)
# Create a 3D surface plot of the cone
fig = go.Figure(data=[go.Surface(x=x_mesh, y=y_mesh, z=z_mesh)])

# Add the original data points to the scatter plot
fig.add_trace(go.Scatter3d(x=Cartesian_Space_noisy[:, 0], y=Cartesian_Space_noisy[:, 1], z=Cartesian_Space_noisy[:, 2], mode='markers', marker=dict(size=2)))

fig.show()

Visually the cone fits quite well onto our data. We can now also extract the relevant cone metrics from this 3D fit 

In [31]:
# Calculate the height of the cone
height = z_max - z_min

# Calculate the radius of the cone
radius = height * abs(r)

# Calculate the apex angle of the cone in degrees
apex_angle = 2 * np.arctan(radius / height) * 180 / np.pi

print(f"-- Cone Parameters --")
print(f'Center: ({a}, {b}, {c})')
print(f'Height: {height}')
print(f'Radius: {radius}')
print(f'Apex Angle: {apex_angle/2}')

-- Cone Parameters --
Center: (0.0036350288287721167, 0.007771731197641822, -0.006777913092642083)
Height: 2.290256101635835
Radius: 1.7965507402040557
Apex Angle: 38.111781071926956


Using mathematical standard ML metrics to judge how good of a fit this cone is we get the following 

In [32]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predict the z coordinates using the fitted cone
z_pred = np.clip(c + r * np.sqrt((Cartesian_Space_noisy[:, 0] - a)**2 + (Cartesian_Space_noisy[:, 1] - b)**2), z_min, z_max)

# Calculate the metrics
rmse = np.sqrt(mean_squared_error(Cartesian_Space_noisy[:, 2], z_pred))
mae = mean_absolute_error(Cartesian_Space_noisy[:, 2], z_pred)
mape = np.mean(np.abs((Cartesian_Space_noisy[:, 2] - z_pred) / Cartesian_Space_noisy[:, 2])) * 100
r2 = r2_score(Cartesian_Space_noisy[:, 2], z_pred)

print("---Metrics---")
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')
print(f'R^2: {r2}')

---Metrics---
RMSE: 0.1887243486433827
MAE: 0.1489977121050541
MAPE: 57.359035719306576%
R^2: 0.858154620289914
